In [160]:
import pandas as pd
test = pd.read_csv('/Users/miji.lee/Downloads/titanic/test.csv')
train = pd.read_csv('/Users/miji.lee/Downloads/titanic/train.csv')
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
# missing values

train.isnull().sum()

In [ ]:
train.duplicated().sum()
# train.drop_duplicates(subset =['PassengerId'], inplace = True)

In [ ]:
train.describe()

In [ ]:
train[train['Age']<1].head()
# dataset[select rows that meet the condition]

In [ ]:
train[(train['Age']>=1)& (train['Age']<=2)].head()
# 복수 조건을 두개 만족해야 할때는 dataset[(select rows with condition 1) & (select rows with condition 2)]

In [ ]:
# training에서 어떤 피쳐가 예측값에 가장 영향을 많이 미치나 파악해야
# 어떤 상관관계를 맺고 있나 -> heatmap (only continuous. category식 데이터는 어려움)
corr_matrix = train.corr(numeric_only= True)
# 문자열 데이터와 같이 숫자가 없는 데이터간 상관관계는 논리적으로 말이 안됨 -> 수치형만 포함시키도록
# 그 수치형이 ID같은 범주형 데이터라도 어느정도 상관관계가 보여짐

corr_matrix
# pearson correlation. 1에 가까우면 한 피쳐가 증가하면 다른 피쳐도 증가. -1은 역관계

In [ ]:
import plotly.express as px

fig_px = px.imshow(corr_matrix, text_auto=True, aspect='auto', 
          title= 'Correlation Heatmap', 
          color_continuous_scale='Blues')

fig_px.show()
#  알고 싶은 것은 survived 이므로 그것과 다른 변수간의 관계를 중점으로 봄
#  fare색상이 진해보임

In [ ]:
# 히트맵으로 안나오는 카테고리형 데이터와 survived의 관계를 알아보자 
survived = train[train['Survived']==1]['Sex'].value_counts()
# survived 칼럼== 1 조건을 만족하는 row number를 뽑아서 train 데이터셋으로 새로운 데이터프레임의 sex칼럼을 뽑음
# 갯수 셈
survived

In [ ]:
dead = train[train['Survived']==0]['Sex'].value_counts()
dead

In [ ]:
df_0 = pd.DataFrame([survived, dead])
df_0
# 만약 'survived', 'dead'처럼 감싸면 
# 0 survived
# 1 dead 처럼 단순한 문자열로 인식.

# survived, dead는 세로로 된 series같이 보이지만 사실은 가로 행으로 구성되있음
# 행데이터들을 pd.DataFrame은 세로로 합친 것

In [ ]:
df_0.index = ['Survived', 'Dead']
# index: 맨 앞 칼럼
df_0

In [ ]:
# 동일하게 concat으로 할수도 있음

df1 = pd.concat([survived, dead], axis =1)
# 1은 옆으로 늘고 0은 위아래로 늘음
df1.columns = ['Survived','Dead']
print(df1)

In [ ]:
result = []  # feature 이름을 key로, DataFrame을 value로
features = ['Sex', 'Pclass']

def fct(feature):
    for feature in features:
        survived =train[train['Survived']==1][feature].value_counts()
        dead = train[train['Survived']==0][feature].value_counts()
        df_graph = pd.DataFrame([survived, dead])
        df_graph.index = ['Survived', 'Dead']
        result.append(df_graph)
        result_df = pd.concat(result, axis =0)
    return result_df

# 함수 실행 후 반환값을 변수에 저장
result_1 = fct(features)


In [ ]:
# 같은 내용을 재활용을 위해 함수로 바꾸면

def visualisation(feature):
    survived =train[train['Survived']==1][feature].value_counts()
    dead = train[train['Survived']==0][feature].value_counts()
    df_graph = pd.DataFrame([survived, dead])
    df_graph.index = ['Survived', 'Dead']
    return df_graph

In [ ]:
df_visual = visualisation('Sex')
df_visual.head()

In [ ]:
# 절대 빈도는 막대 그래프 (남자에 비해 여자가 더 많이 살았다)
# 상대 빈도는 파이 차트 (여자 중 사망한 %)

fig  = px.bar(df_0,
       x = df_0.index,
       y = df_0.columns,
       barmode = 'group',
        text_auto = True,
       title ='Survived Count by Sex')

fig.show()

In [ ]:
df_pc = visualisation('Pclass')
df_pc

In [ ]:
fig = px.bar(df_pc,
       x=df_pc.index,
       y=df_pc.columns,
       barmode ='stack',
       text_auto = True,
       title ='Survived Count by Class')
fig.show()

In [ ]:
# 등급을 x축에 놓고 등급별로 확인하려면 plotly 특성상 x축은 인덱스만 쓸 수 있음
# 지금 인덱스는 생존 여부이므로 transpose

df_transpose = df_pc.transpose()
df_transpose

In [ ]:
fig = px.bar(df_transpose,
       x=df_transpose.index,
       y=df_transpose.columns,
       barmode ='stack',
       text_auto = True,
       title ='Survived Count by Class')
fig.show()